## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


Import necessary libraries

In [514]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Import data into dataframe

In [515]:
basic_df = pd.read_csv(r'C:\Users\Sylvia\moringa_labs\project1\dsc_project\zippedData\imdb.title.basics.csv.gz')
rating_df = pd.read_csv(r'C:\Users\Sylvia\moringa_labs\project1\dsc_project\zippedData\imdb.title.ratings.csv.gz')
gross_df = pd.read_csv(r'C:\Users\Sylvia\moringa_labs\project1\dsc_project\zippedData\bom.movie_gross.csv.gz')
tn_budget_df = pd.read_csv(r'C:\Users\Sylvia\moringa_labs\project1\dsc_project\zippedData\tn.movie_budgets.csv.gz')
rt_movies_df = pd.read_csv(r'C:\Users\Sylvia\moringa_labs\project1\dsc_project\zippedData\rt.movie_info.tsv.gz', sep="\t")
# rt_ratings_df = pd.read_csv(r'C:\Users\Sylvia\moringa_labs\project1\dsc_project\zippedData\rt.reviews.tsv.gz', sep="\t")

Overview of the datasets

In [516]:
basic_df.info()
basic_df.head(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


Check for duplicates

In [517]:
basic_df.duplicated().value_counts()

False    146144
dtype: int64

Clean basic_df. Drop original_title, it is repetitive and renaming primary_title to movie_title for clarity.

In [518]:
basic_df.drop(columns=['original_title'], inplace=True)
basic_df.rename({'primary_title':'movie_title'}, axis=1, inplace=True)
basic_df.head()

,tconst,movie_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy"


Check %age of Nans to handle missing values


In [519]:
basic_df.isna().sum()*100/len(basic_df)


tconst              0.000000
movie_title         0.000000
start_year          0.000000
runtime_minutes    21.717621
genres              3.700460
dtype: float64

We have 23% Nan values. Let's fill it with mode. Then change runtime to int type

In [520]:
basic_df['runtime_minutes'].mode()
basic_df['runtime_minutes'].fillna(90, inplace=True)
basic_df['runtime_minutes'] = basic_df['runtime_minutes'].astype(int)
# basic_df = basic_df({'runtime_minutes'}).astype(int)
basic_df.head(5)

,tconst,movie_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,2013,175,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,2019,114,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,2018,122,Drama
3,tt0069204,Sabse Bada Sukh,2018,90,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,2017,80,"Comedy,Drama,Fantasy"


Splitting to find the dominant genre of the movies for categorical visualization.

In [521]:
basic_df['genres'] = basic_df['genres'].str.split(',', expand=True)
basic_df['genres'].fillna('Unknown', inplace=True)
basic_df.head()
basic_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   tconst           146144 non-null  object
 1   movie_title      146144 non-null  object
 2   start_year       146144 non-null  int64 
 3   runtime_minutes  146144 non-null  int32 
 4   genres           146144 non-null  object
dtypes: int32(1), int64(1), object(3)
memory usage: 5.0+ MB


Overview of gross_df

In [522]:
gross_df.info()
gross_df.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


,title,studio,domestic_gross,foreign_gross,year
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018
3386,An Actor Prepares,Grav.,1700.0,NaN,2018


In [523]:
gross_df.isnull().sum()/len(gross_df)*100

title              0.000000
studio             0.147623
domestic_gross     0.826690
foreign_gross     39.858282
year               0.000000
dtype: float64

In [524]:
gross_df.duplicated().value_counts()

False    3387
dtype: int64

Drop the Nan rows. 

In [525]:
gross_df.dropna(inplace=True)

Change foreign_gross to float to perform math operations

In [526]:
gross_df['foreign_gross'] = pd.to_numeric(gross_df['foreign_gross'].str.replace(',', '', regex=True))
gross_df.info()
# gross_

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2007 entries, 0 to 3353
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           2007 non-null   object 
 1   studio          2007 non-null   object 
 2   domestic_gross  2007 non-null   float64
 3   foreign_gross   2007 non-null   float64
 4   year            2007 non-null   int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 94.1+ KB


Pick top 50 performing movies.

In [527]:
gross_df_sorted = gross_df.sort_values(by='domestic_gross', ascending=False)
gross_df_sorted.head(5)

,title,studio,domestic_gross,foreign_gross,year
1872,Star Wars: The Force Awakens,BV,936700000.0,1131.6,2015
3080,Black Panther,BV,700100000.0,646900000.0,2018
3079,Avengers: Infinity War,BV,678800000.0,1369.5,2018
1873,Jurassic World,Uni.,652300000.0,1019.4,2015
727,Marvel's The Avengers,BV,623400000.0,895500000.0,2012


In [528]:
rating_df.info()
rating_df.head(5)
# rating_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


Check for duplicates

In [529]:
rating_df.duplicated().value_counts()

False    73856
dtype: int64

In [530]:
tn_budget_df.info()
tn_budget_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


Check for duplicates

In [531]:
tn_budget_df.duplicated().value_counts()

False    5782
dtype: int64

Create month and year columns for categorical comparison. Then drop release date

In [532]:
tn_budget_df['release_month'] = tn_budget_df['release_date'].str.split(' ').str[0]
tn_budget_df['release_year'] = tn_budget_df['release_date'].str.split(' ').str[-1]
tn_budget_df = tn_budget_df.drop(columns='release_date', axis=0)


tn_budget_df.head()

,id,movie,production_budget,domestic_gross,worldwide_gross,release_month,release_year
0,1,Avatar,"$425,000,000","$760,507,625","$2,776,345,279",Dec,2009
1,2,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",May,2011
2,3,Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",Jun,2019
3,4,Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",May,2015
4,5,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",Dec,2017


In [533]:

# tn_budget_df = tn_budget_df.astype({'production_budget':'int', 'domestic_gross':'int','worldwide_gross':'int'})

tn_budget_df['domestic_gross'] = tn_budget_df['domestic_gross'].str.replace(r"[$,]", "", regex=True).astype(int)
tn_budget_df['production_budget'] = tn_budget_df['production_budget'].str.replace(r"[$,]", "", regex=True).astype(int)
# tn_budget_df['worldwide_gross'] = tn_budget_df['worldwide_gross'].str.replace(r"[$,]", "", regex=True).astype(int) # to big to execute

tn_budget_df["worldwide_gross"] = pd.to_numeric(tn_budget_df["worldwide_gross"].str.replace(r"[$,]", "", regex=True))#

tn_budget_df.info()
tn_budget_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   movie              5782 non-null   object
 2   production_budget  5782 non-null   int32 
 3   domestic_gross     5782 non-null   int32 
 4   worldwide_gross    5782 non-null   int64 
 5   release_month      5782 non-null   object
 6   release_year       5782 non-null   object
dtypes: int32(2), int64(2), object(3)
memory usage: 271.2+ KB


,id,movie,production_budget,domestic_gross,worldwide_gross,release_month,release_year
0,1,Avatar,425000000,760507625,2776345279,Dec,2009
1,2,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,May,2011
2,3,Dark Phoenix,350000000,42762350,149762350,Jun,2019
3,4,Avengers: Age of Ultron,330600000,459005868,1403013963,May,2015
4,5,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,Dec,2017


Don't need original_title
Clean up above before continuing